In [24]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('train.csv')
dt = pd.read_csv('test.csv')
sf= pd.read_csv('sample_submission.csv')

In [3]:
df.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [4]:
df = df.drop('id', axis=1)
df = df.drop('Policy Start Date', axis=1)
dt = dt.drop('id', axis=1)
dt = dt.drop('Policy Start Date', axis=1)


In [5]:
df.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,Poor,Yes,Weekly,House,2022.0


In [6]:
for label, content in df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df[label] = content.fillna(content.mean())
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df[label] = content.astype("category").cat.add_categories("unspecified").fillna("unspecified")

In [7]:
for label, content in dt.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            dt[label] = content.fillna(content.mean())
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            dt[label] = content.astype("category").cat.add_categories("unspecified").fillna("unspecified")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 19 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   Age                   1200000 non-null  float64 
 1   Gender                1200000 non-null  object  
 2   Annual Income         1200000 non-null  float64 
 3   Marital Status        1200000 non-null  category
 4   Number of Dependents  1200000 non-null  float64 
 5   Education Level       1200000 non-null  object  
 6   Occupation            1200000 non-null  category
 7   Health Score          1200000 non-null  float64 
 8   Location              1200000 non-null  object  
 9   Policy Type           1200000 non-null  object  
 10  Previous Claims       1200000 non-null  float64 
 11  Vehicle Age           1200000 non-null  float64 
 12  Credit Score          1200000 non-null  float64 
 13  Insurance Duration    1200000 non-null  float64 
 14  Customer Feedback 

In [9]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   Age                   800000 non-null  float64 
 1   Gender                800000 non-null  object  
 2   Annual Income         800000 non-null  float64 
 3   Marital Status        800000 non-null  category
 4   Number of Dependents  800000 non-null  float64 
 5   Education Level       800000 non-null  object  
 6   Occupation            800000 non-null  category
 7   Health Score          800000 non-null  float64 
 8   Location              800000 non-null  object  
 9   Policy Type           800000 non-null  object  
 10  Previous Claims       800000 non-null  float64 
 11  Vehicle Age           800000 non-null  float64 
 12  Credit Score          800000 non-null  float64 
 13  Insurance Duration    800000 non-null  float64 
 14  Customer Feedback     800000 non-nul

In [10]:
df.isnull().sum()

,0
Age,0
Gender,0
Annual Income,0
Marital Status,0
Number of Dependents,0
Education Level,0
Occupation,0
Health Score,0
Location,0
Policy Type,0


In [11]:
dt.isnull().sum()

,0
Age,0
Gender,0
Annual Income,0
Marital Status,0
Number of Dependents,0
Education Level,0
Occupation,0
Health Score,0
Location,0
Policy Type,0


In [12]:
X = df.drop("Premium Amount", axis=1)
y = df["Premium Amount"]

In [13]:
cats = X.select_dtypes(exclude=np.number).columns.tolist()
for col in cats:
   X[col] = X[col].astype('category')

In [14]:
X, y = df.drop('Premium Amount', axis=1), df['Premium Amount']

In [15]:
categorical_features = X.select_dtypes(
   include=["object"]
).columns.tolist()

numerical_features = X.select_dtypes(
   include=["float64", "int64"]
).columns.tolist()

In [19]:
preprocessor = ColumnTransformer(
   transformers=[
       ("cat", OneHotEncoder(), categorical_features),
       ("num", StandardScaler(), numerical_features),
   ]
)


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 19 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   Age                   1200000 non-null  float64 
 1   Gender                1200000 non-null  object  
 2   Annual Income         1200000 non-null  float64 
 3   Marital Status        1200000 non-null  category
 4   Number of Dependents  1200000 non-null  float64 
 5   Education Level       1200000 non-null  object  
 6   Occupation            1200000 non-null  category
 7   Health Score          1200000 non-null  float64 
 8   Location              1200000 non-null  object  
 9   Policy Type           1200000 non-null  object  
 10  Previous Claims       1200000 non-null  float64 
 11  Vehicle Age           1200000 non-null  float64 
 12  Credit Score          1200000 non-null  float64 
 13  Insurance Duration    1200000 non-null  float64 
 14  Customer Feedback 

In [25]:
pipeline = Pipeline(
   [
       ("preprocessor", preprocessor),
       ("classifier", XGBRegressor (random_state=42, max_depth=4,  learning_rate=0.01,  n_estimators= 500,))
   ]
)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=13
)

In [27]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['Gender', 'Education Level',
                                                   'Location', 'Policy Type',
                                                   'Smoking Status',
                                                   'Exercise Frequency',
                                                   'Property Type']),
                                                 ('num', StandardScaler(),
                                                  ['Age', 'Annual Income',
                                                   'Number of Dependents',
                                                   'Health Score',
                                                   'Previous Claims',
                                                   'Vehicle Age',
                                                   'Credit Score',
                                                   'Insurance Duration'])])),
                ('classif...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=500, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [28]:
y_pred = pipeline.predict(X_test)

In [29]:
categorical_features = dt.select_dtypes(
   include=["object"]
).columns.tolist()

numerical_features = dt.select_dtypes(
   include=["float64", "int64"]
).columns.tolist()

In [30]:
preprocessor = ColumnTransformer(
   transformers=[
       ("cat", OneHotEncoder(), categorical_features),
       ("num", StandardScaler(), numerical_features),
   ]
)


In [31]:
id = sf.pop('id')
y_pred = pipeline.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'Premium Amount': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
